# Lecture 3: Password Attack with Differential-Power-Analysis (Kocher et al. 1999)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("sbox_lookup.c"))
capture_device.flash()

In [ ]:
data = capture_device.capture(
    number_of_traces=1000,
    input=lambda _: [random.randint(0, 255)] + 15 * [0],
)

In [ ]:
def selection_function(value, guess):
    return lascar.tools.aes.sbox[value["input"][0] ^ guess] & 0x80 == 0


trace = lascar.TraceBatchContainer(data["trace"], data)
engine = lascar.DpaEngine(
    selection_function=selection_function,
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=lascar.TableOutputMethod(engine),
)
session.run(batch_size="auto")